In [34]:
import csv
import sqlite3
# from flask import jsonify
import json
# import pandas as pd


In [13]:
def connect(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    return conn, cursor


In [3]:
def insert_some_data():
    try:
        with open('clustered_articles_small.csv', 'r') as data:
            reader = csv.DictReader(data)
            to_db = [(i['article_id'], i["Keywords"], i['Title'], i["Text"], i["Bias"], i['Cluster Tags']) for i in reader]

        cursor.executemany("INSERT INTO Articles (article_id, Title, Text, Bias, Keywords, [Cluster Tags]) VALUES (?, ?, ?, ?, ?, ?);", to_db)
    #     con.commit()
    #     cur.close()
    except sqlite3.Error as error:
        print("Error: ", error)
    # finally:
    #     con.close()
    #     print("sqlite connection closed")

In [32]:
def get_histogram_data():
    # cluster_tags = [
    #     "Political and Social Issues",
    #     "Community Involvement",
    #     "Names, Organizations, and Various Terms",
    #     "Locations",
    #     "Misc. Adjectives",
    #     "Legal and Law Enforcement",
    #     "Science and Medicine",
    #     "Noise/Nonsensical Phrases",
    #     "Media and Entertainment",
    #     "Various Phrases/Verbs",
    #     "Nature and Wildlife",
    #     "Food",
    #     "Finance and Economics",
    #     "Objects and Accessories",
    # ]
    conn = sqlite3.connect('first_2000.db')
    conn = conn.cursor()
    conn.execute(
        """
        SELECT cluster_id, cluster_name from clusters;"""
    )
    cluster_tags = conn.fetchall()
    cluster_tags = [tag[1] for tag in cluster_tags[1:]]
    
    # this one uses small.db, created this from shell
    '''
    sqlite3 small.db
    .mode csv articles
    .import clustered_articles_small.csv articles  # this is the first 2000 rows of large csv
    .tables  # prints 'articles'
    .schema  # also gives more info
    .quit
    '''
    conn.execute("Select bias, count(*) from articles where [Cluster Tags] like '%Nature And WildLife%' group by bias")
    data = conn.fetchall()
    print(data)

    # conn = sqlite3.connect('small.db')
    # conn = conn.cursor()


    # histogram_data = []
    # for tag in cluster_tags:
    #     print("current tag", tag)
    #     like_tag = "%" + tag + "%"
    #     conn.execute("""
    #         SELECT bias, count(*) AS article_count FROM articles 
    #         WHERE [Cluster Tags] LIKE ?
    #         """, (like_tag,))
    #     cluster_data = conn.fetchall()
    #     print(cluster_data)


        # histogram_data.append({
        #     "tag": tag,
        #     "data": cluster_data
        # })
    # conn.close()

    return json.dumps(data)

In [33]:
get_histogram_data()

[('0', 12), ('1', 11), ('2', 30), ('3', 1), ('4', 18)]


'[["0", 12], ["1", 11], ["2", 30], ["3", 1], ["4", 18]]'

In [67]:
def get_articles_in_cluster(clusterId=0, limit=50):

    conn = sqlite3.connect('first_2000.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    # conn.execute("SELECT * FROM articles")
    cursor.execute("""
        SELECT cluster_name from clusters where cluster_id = ?;
                 """, (clusterId,)
    )
    cluster_tags = cursor.fetchall()
    print("cluster_tags", cluster_tags)
    if len(cluster_tags) != 1:
        print("wrong number of tags returned!")
        return None
    tag_name = cluster_tags[0][0]
    print(tag_name)
    # cluster_tag = [tag[1] for tag in cluster_tag[1:]]
    tag_name = "%" + tag_name + "%"
    cursor.execute(
        f"""SELECT article_id, Bias, Title, Text, Keywords, [Cluster Tags]
            FROM articles
            WHERE [Cluster Tags] like ? limit ?;""", (tag_name,limit,)
    )
    articles = cursor.fetchall()
    print("articles", articles)
    conn.close()

    return json.dumps([dict(article) for article in articles])

In [68]:
get_articles_in_cluster(1, 5)

cluster_tags [<sqlite3.Row object at 0x106ba9480>]
Community Involvement
articles [<sqlite3.Row object at 0x106baace0>, <sqlite3.Row object at 0x106ba9ff0>, <sqlite3.Row object at 0x106ba94b0>, <sqlite3.Row object at 0x106bab490>, <sqlite3.Row object at 0x106ba82e0>]


'[{"article_id": 1, "Bias": "0", "Title": "University To Award Trayvon Martin With Posthumous Degree In Aviation", "Text": "A Florida university will honor Trayvon Martin with a posthumous Bachelor of Science Degree in Aviation at a May 13 commencement ceremony. The degree will be accepted by his parents, Tracy Martin and Sabrina Fulton. Seventeen-year-old Martin was killed in February of 2012 by former neighborhood watch captain George Zimmerman, who is Hispanic, in self-defense. After a long, racially-charged public debate and court trial, Zimmerman was completely exonerated in July of 2013. Florida Memorial University officially announcedthe posthumous degree via Facebook on Wednesday. \\"The University will confer upon TRAYVON MARTIN a posthumous degree in Aeronautical Science with a concentration in Flight Education, in honor of the steps he took during his young life toward becoming a pilot,\\" says the Facebook post. \\"This particular Bachelor\\u2019s degree is designated for t